In [ ]:
!pip3 uninstall -y spacy 
!pip3 install spacy
!python3 -m spacy download ru_core_news_sm
!python3 -m spacy download en

Found existing installation: spacy 2.2.4
Uninstalling spacy-2.2.4:
  Successfully uninstalled spacy-2.2.4
     |████████████████████████████████| 6.2 MB 5.3 MB/s 
     |████████████████████████████████| 42 kB 1.1 MB/s 
     |████████████████████████████████| 653 kB 45.4 MB/s 
     |████████████████████████████████| 10.1 MB 40.5 MB/s 
     |████████████████████████████████| 457 kB 36.5 MB/s 
     |████████████████████████████████| 181 kB 44.3 MB/s 
     |████████████████████████████████| 58 kB 4.3 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.2.0
    Uninstalling typing-extensions-4.2.0:
      Successfully uninstalled typing-extensions-4.2.0
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstal

In [ ]:
import re
import spacy
from spacy.tokenizer import Tokenizer
spacyRu = spacy.load('ru_core_news_sm')

import string
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.options.display.max_colwidth = 100
import csv
import json
import glob
from nltk.tokenize import WordPunctTokenizer

from google.colab import drive
drive.mount('/content/drive')
!pip install stanza nltk

import stanza
stanza.download('ru')

import nltk
from nltk.tokenize import sent_tokenize 
nltk.download('punkt')
nlp = stanza.Pipeline(lang='ru', processors='tokenize')
import difflib


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


2022-05-06 16:24:08 INFO: Downloading default packages for language: ru (Russian)...
2022-05-06 16:24:11 INFO: File exists: /root/stanza_resources/ru/default.zip
2022-05-06 16:24:18 INFO: Finished downloading models and saved to /root/stanza_resources.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


2022-05-06 16:24:19 INFO: Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |

2022-05-06 16:24:19 INFO: Use device: cpu
2022-05-06 16:24:19 INFO: Loading: tokenize
2022-05-06 16:24:19 INFO: Done loading processors!


In [ ]:
import os
import sys
REPO_PATH = "/content/drive/MyDrive/Diploma_paper/mrc_preprocess_wanglaiki/mrc-wanglaiki"
os.environ['PYTHONPATH'] += ":" + REPO_PATH
sys.path.insert(1, REPO_PATH)

In [ ]:
forbidden_symbols = {'\t':' ', '\r':' ', '\f':' ', '\xa0':' ', '\n':' ', '=—':'  ', "\|":" ",'サ':" ",
 'ミ':" ", 'ヲ':" ", '一':" ", '三':" ", '中':" ", '丹':" ", '二':" ", '京':" ", '代':" ", '何':" ", '作':" ", '先':" ",
 '凌':" ", '前':" ", '友':" ", '右':" ", '君':" ", '咪':" ", '国':" ", '大':" ", '字':" ", '富':" ", '少':" ", '川':" ",
 '市':" ", '年':" ", '後':" ", '恵':" ", '木':" ", '村':" ", '次':" ", '浦':" ", '澤':" ", '猷':" ", '琳':" ", '章':" ",
 '美':" ", '聯':" ", '莹':" ", '蒙':" ", '衛':" ", '袁':" ", '郎':" ", '里':" ", '锋':" ", '門':" ", '队':" ", '雄':" ", '頭':" ", '颖':" ", '马':" ",
 'É':' ', '×':' ', 'á':' ', 'ä':' ', 'ç':' ', 'è':' ', 'é':' ', 'í':' ', 'ó':' ', 'ô':' ', 'ú':' ',
'ü':' ', 'ć':' ', 'Š':' ', 'š':' ', 'ʼ':' ', '́':' ', 'є':' ', 'ғ':' ', 'ב':' ',
'ד':" ",
'ה':' ',
'ו':' ',
'ח':' ',
'י':' ',
'ל':' ',
'ם':' ',
'ס':' ',
'פ':' ',
'צ':' ',
'ר':' ',
'ת':' ',
'ا':' ',
'ح':' ',
'ر':' ',
'ف':' ',
'ل':' ',
'م':' ',
'ن':' ',
'و':' ',
'ی':' ', '\u2002':' ', '\u2009':' ', '\u200b':' ', '\u202f':' ', '▶':' ',
'«\'\'':' «\'', '\'\'»':'\'» ',
"(''": "( '", "'')": "' )", 
"—":"-"
}

emoticons = r'(?:[:;=\^\-oO][\-_\.]?[\)\(\]\[\-DPOp_\^\\\/])'

regex_tweets = [
    emoticons,
    r'<[^>]+>',      ## HTML TAGS
    r'(?:@[\w\d_]+)',   ## @-mentions
    r'(?:\#[\w]+)',  ## #HashTags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
    r'\d{4}(?=[-]\d{4})', #years
    r'(?<=\d{4}[-])\d{4}', # years
    r'(?=\b[MCDXLVI]{1,6}\b)M{0,4}(?:CM|CD|D?C{0,3})(?:XC|XL|L?X{0,3})(?:IX|IV|V?I{0,3})', # roman numbers
    # r"(?<=[а-яА-Яa-zA-Z]-)?(?:\d*\.\d+|\d+)", # numbers Мурманск-198 => 198
    # r"(?<=[^а-яА-Яa-zA-Z])[-+]?(?:\d*\.\d+|\d+)", #numbers positive/negative/Мурманск-198
    r"(?<=[^а-яА-Яa-zA-Z0-9»])[-+]?(?:\d*\.\d+\.\d+\.\d+|\d*\.\d+\.\d+| \d*\.\d+|\d+)", #numbers positive/negative/Мурманск-198/13.3.4.3/12.3.4
    r"^[-+]?(?:\d*\.\d+\.\d+\.\d+|\d*\.\d+\.\d+| \d*\.\d+|\d+)", #newline of above
    # r"[-+]?(?:\d*\.\d+|\d+)", # numbers even negative
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    # r'(?:(?:\d+,?)+(?:\.?\d+)?)' ,  ##numbers 
    r'(?:[\w_]+)',   #other words
    r'(?:\S)'        ## normal text 
]

#compiling regex
tokens_re = re.compile(r'('+'|'.join(regex_tweets)+')' , re.IGNORECASE | re.VERBOSE)

spacyRu.tokenizer = Tokenizer(spacyRu.vocab, infix_finditer=tokens_re.finditer)
# (?<=[а-яА-Яa-zA-Z]-)\d+ 

In [ ]:
def word_tokenizer_span(text, method='spacy'):
  words = [] 
  spans = []
  if method == 'spacy':
    # doc = spacyRu(text)
    # words = [t.text for t in doc]
    # spans = [(token.idx, token.idx + len(token)) for token in doc]
    words = tokens_re.findall(text)
    spans = [x.span() for x in tokens_re.finditer(text)]
  if method == 'nltk':
    words = WordPunctTokenizer().tokenize(text)
    spans = list(WordPunctTokenizer().span_tokenize(text))
  return words, spans
  

def sentence_tokenizer(text, method):
    res = []
    if method == 'nltk':
        res = sent_tokenize(text)
    elif method == 'stanza':
        model = stanza.Pipeline(lang='ru', processors='tokenize')
        doc = model(text)
        res = [sentence.text for sentence in doc.sentences]
    elif method == 'spacy':
        model = spacy.load('ru_core_news_sm')
        tokens = model(text)
        # res = [sent for sentence in tokens.sents]
        for sent in tokens.sents:
            res.append(sent)
            # print(sent)
    return res

# indices - [a,b] is a letter index, iwords - letter indices for words.
def give_word_index(indices, iwords, take_two_spans):
    res = []
    for i, ind in enumerate(indices):
      if i % 2 == 0:
          tmp = []
          for j, w in enumerate(iwords):
              if w[0] >= indices[i] and w[1] <= indices[i+1]:
                  tmp.append(j)
          if len(tmp) > 0:
            res += [tmp[0], tmp[-1]]
    if take_two_spans:
      try:
        return [res[0], res[-1]]
      except:
        return [] # it means string is removed since it has forbidden symbols (Chiness?)
    else:
      return res

    
def give_me_word(word_ind, words):
    res = []
    for i, ind in enumerate(word_ind):
        if i % 2 == 0:
            res += words[word_ind[i] : word_ind[i+1]+1]
    return res



def give_me_text(indices, text):
    res =''
    # print(indices)
    for i, ind in enumerate(indices):
        if i % 2 == 0:
            res += text[indices[i]:indices[i+1]]
    return res

def give_me_new_index(old_index_list, map_old_index2new_index):
  res = [map_old_index2new_index[old_index] for old_index in old_index_list]
  # print(old_index_list, res)
  return res

def remove_forbidden_symbols(text, forbidden_symbols):
  text_new = str(text)
  for fs, sub in forbidden_symbols.items():
      text_new = re.sub(re.escape(fs), sub, text_new)
  return text_new

def generate_map_old_to_new_text(text, text_new, forbidden_symbols):
  symbols = forbidden_symbols.keys()
  new_index = 0
  old_index = 0
  map_old_index2new_index = {} # old_Id: new_Id

  for old_index, s in enumerate(text):
    map_old_index2new_index[old_index] = new_index
    new_index += 1
    # if s in symbols:
    #   if old_index < len(text):
    #     if text[old_index + 1] not in symbols:
    #       new_index += 1
    #   continue
    # else:
    #   if new_index < len(text_new):
    #     if s == text_new[new_index]:
    #       new_index += 1
  map_old_index2new_index[len(text)] = len(text_new)
  return map_old_index2new_index


def check(df, forbidden_symbols):
  for index, row in df.iterrows():
    etalon = row['entity'].replace('\xa0', '').replace(' ', '')
    if set(etalon).intersection(forbidden_symbols.keys()) or len(row['num']) > 2:
      continue
    extracted = row['extracted'].replace(' ', '')
    words_extracted_by_wi = ''.join(row['words_extracted_by_wi'])
    s = ''.join(row['words_extracted_by_wi'])
    ff = False
    # if etalon != extracted:
    #   print(f"Warning: extracted is different from entity=\'{repr(etalon)}\', extracted=\'{repr(extracted)}\'. Differ={etalon != extracted}")
    #   output_list = [li for li in difflib.ndiff(row['entity'], row['extracted']) if li[0] != ' ']
    #   print(f"output_list={output_list}")
    #   ff = True
    if words_extracted_by_wi != etalon:
      print(f"Words_extracted_by_wi:\'{repr(words_extracted_by_wi)}\'; etalon:\'{repr(etalon)}\'")
      ff = True
    if ff:
      print('************')
      print(row)
      print('------------')
def remove_row(df, row):
  df = df[df['num'] != row]
  return df

In [ ]:
def preprocess_one_file(path, release=True, sentence_delimiter=True, take_two_spans=True):

    pandas_csv = pd.read_csv(path+'ann', delimiter='\t', header=None, names=['type', 'label_and_num', 'entity'])
    
    df = pandas_csv.drop(pandas_csv.index[pandas_csv['type'].str.startswith('R')])
    try:
      tmp = pd.DataFrame(df['label_and_num'].str.split(' ',1).tolist(),
                      columns = ['label','num'], index=df.index)

      tmp['num'] = tmp['num'].str.split(' |;').apply(lambda x: [int(i) for i in x])
      df['label'] = tmp['label']
      df['num'] = tmp['num']
    except:
      pass

    if release:
      df.drop(labels=['type', 'label_and_num'], axis=1, inplace=True)

    # open context file
    with open(path+'txt') as f:
      text = f.read()

    # text1 = remove_forbidden_symbols(text, forbidden_symbols)
    # text_new = remove_forbidden_symbols(text1, {'     ':' ', '    ':' ', '   ':' ', '  ':' '})
    text_new = remove_forbidden_symbols(text, forbidden_symbols)
    if not release:
      print('OLD TEXT:')
      display(repr(text))
      print(f'NEW TEXT after removing {forbidden_symbols}')
      display(repr(text_new))

    map_old_index2new_index = generate_map_old_to_new_text(text, text_new, forbidden_symbols)
    text = str(text_new)

    if not release:
      print('MAP of OLD TEXT INDICES to NEW TEXT:')
      print(map_old_index2new_index)

    df['num'] = df['num'].apply(lambda x: give_me_new_index(x, map_old_index2new_index))

    words, iwords = word_tokenizer_span(text, method='spacy')
    # print('words', words)
    df['new_span'] = df['num'].apply(lambda x: give_word_index(x, iwords, take_two_spans))
    for index, row in df.iterrows():
      row_new_span = row['new_span']
      if set(row['entity']).intersection(forbidden_symbols.keys()) or not row_new_span:
        continue

      N_found = row_new_span[-1] - row_new_span[0] + 1
      N_etalon = len(word_tokenizer_span(row['entity'], method='spacy')[1])
      if N_found < N_etalon:
        row_new_span[-1] += 1
        row['new_span'] = row_new_span
        print(f"NEREL NER error: {row}, N_found={N_found}, N_etalon={N_etalon}" )

    
    df['extracted'] = df['num'].apply(lambda x: give_me_text(x, text))
    df['words_extracted_by_wi'] = df['new_span'].apply(lambda x: give_me_word(x, words))
    if not release:
      display(df)
      print('Words', repr(words))
    check(df, forbidden_symbols)

    if sentence_delimiter:
      res = []
      s_delim_space_all = ""
      offset = 0
      doc = nlp(text)
      for sentence in doc.sentences:
          # if not release:
          #   print('sentence.tokens len:', len(sentence.tokens), 'sentence:', sentence.tokens)
          sent_span = (sentence.tokens[0].start_char, sentence.tokens[-1].end_char) # end_char not inclusive
          words_by_sentence, iwords_by_sentence = word_tokenizer_span(text[sent_span[0]:sent_span[1]], method='spacy')
          # print('len(words_by_sentence)=', len(words_by_sentence), 'words_by_sentence', words_by_sentence)
          ans = {'context': " ".join(words_by_sentence)}
            
          d = {} 
          for index, row in df.iterrows():
              key, value = row['label'], row['new_span']
              if value:
                if row['num'][0] >= sent_span[0] and row['num'][-1] < sent_span[1]:# ? не включительно
                    d.setdefault(key, [])
                    if not release:
                      print('key:', key, ' value:', value)
                      print('text:', repr(ans['context']), '[start,end):', (value[0] - offset), (value[-1] - offset + 1))
                    try:
                      if not release:
                        print('Check[', key,'], words_by_sentence[newspan-offset]:', words_by_sentence[(value[0] - offset):(value[-1] - offset + 1)], " words[new_span]", words[(value[0]):(value[-1] + 1)])
                      if words_by_sentence[(value[0] - offset):(value[-1] - offset + 1)] != words[(value[0]):(value[-1] + 1)]:
                        print(f"Error not equal results words_by_sentence:{words_by_sentence[(value[0] - offset):(value[-1] - offset + 1)]}, words: {words[(value[0]):(value[-1] + 1)]}")
                    except:
                      print('Error')
                    if take_two_spans and len(value) > 0:
                      d[key].append( str(value[0] - offset) + ',' + str(value[-1] - offset) )
                    else:
                      for i, ind in enumerate(value):
                          if i % 2 == 0:
                              d[key].append( str(value[i] - offset) + ',' + str(value[i+1] - offset) )
          ans['label'] = d
          res.append(ans)
          offset += len(words_by_sentence)
          if not release:
            print(f'offset={offset}, len_s={len(words_by_sentence)}')
      return res
    else:
      res = {}
      res ['context'] = " ".join(words)
      d = {}
      for index, row in df.iterrows():
        key, value = row['label'], row['new_span']
        d.setdefault(key, [])
        if take_two_spans and len(value) > 0:
            d[key].append( str(value[0]) + ',' + str(value[-1]))
        else:
          for i, ind in enumerate(row['new_span']):
            if i % 2 == 0:
              d[key].append( str(value[i]) + ',' + str(value[i+1]))
      res['label'] = d
      return res
    # except:
    #   return {'context':"", 'label':{}}

In [ ]:
word_tokenizer_span("a -13.15.2 d 1920-1230 cdsc 1980 III-VII III", method='spacy')

(['a',
  '-13.15.2',
  'd',
  '1920',
  '-',
  '1230',
  'cdsc',
  '1980',
  'III',
  '-',
  'VII',
  'III'],
 [(0, 1),
  (2, 10),
  (11, 12),
  (13, 17),
  (17, 18),
  (18, 22),
  (23, 27),
  (28, 32),
  (33, 36),
  (36, 37),
  (37, 40),
  (41, 44)])

In [ ]:
dict_one_file = preprocess_one_file('/content/drive/MyDrive/Diploma_paper/NEREL_01/train/37695_text.', 
                    release=False,
                    # release=True,
                    sentence_delimiter=True,
                    take_two_spans = True)
# dev/598801_text.
# train/157795_text.

OLD TEXT:


"'Сурков подал в отставку\\nВладислав Сурков\\n8 мая 2013 года Президент России Владимир Путин подписал Указ «О Суркове В. Ю.», которым отправил чиновника в отставку:\\n В соответствии с пунктом д статьи 83 Конституции Российской Федерации освободить Суркова Владислава Юрьевича от должности Заместителя Председателя Правительства Российской Федерации Руководителя Аппарата Правительства Российской Федерации по собственному желанию \\n\\nПресс-секретарь президента Дмитрий Песков сообщил, что Сурков подал в отставку после совещания с главой государства по исполнению майских указов:\\nРешение президента о подписании заявления связано с тематикой первоочередных задач по реализации майских указов президента и предвыборных обещаний, и функционировании в Белом доме комиссии, которая занималась реализацией этих указов.\\n\\nПричём в комментариях СМИ он подчёркивал, что Сурков ушёл со своего поста по собственному желанию:\\nПрезидент удовлетворил просьбу Суркова об увольнении по собственному жела

NEW TEXT after removing {'\t': ' ', '\r': ' ', '\x0c': ' ', '\xa0': ' ', '\n': ' ', '=—': '  ', '\\|': ' ', 'サ': ' ', 'ミ': ' ', 'ヲ': ' ', '一': ' ', '三': ' ', '中': ' ', '丹': ' ', '二': ' ', '京': ' ', '代': ' ', '何': ' ', '作': ' ', '先': ' ', '凌': ' ', '前': ' ', '友': ' ', '右': ' ', '君': ' ', '咪': ' ', '国': ' ', '大': ' ', '字': ' ', '富': ' ', '少': ' ', '川': ' ', '市': ' ', '年': ' ', '後': ' ', '恵': ' ', '木': ' ', '村': ' ', '次': ' ', '浦': ' ', '澤': ' ', '猷': ' ', '琳': ' ', '章': ' ', '美': ' ', '聯': ' ', '莹': ' ', '蒙': ' ', '衛': ' ', '袁': ' ', '郎': ' ', '里': ' ', '锋': ' ', '門': ' ', '队': ' ', '雄': ' ', '頭': ' ', '颖': ' ', '马': ' ', 'É': ' ', '×': ' ', 'á': ' ', 'ä': ' ', 'ç': ' ', 'è': ' ', 'é': ' ', 'í': ' ', 'ó': ' ', 'ô': ' ', 'ú': ' ', 'ü': ' ', 'ć': ' ', 'Š': ' ', 'š': ' ', 'ʼ': ' ', '́': ' ', 'є': ' ', 'ғ': ' ', 'ב': ' ', 'ד': ' ', 'ה': ' ', 'ו': ' ', 'ח': ' ', 'י': ' ', 'ל': ' ', 'ם': ' ', 'ס': ' ', 'פ': ' ', 'צ': ' ', 'ר': ' ', 'ת': ' ', 'ا': ' ', 'ح': ' ', 'ر': ' ', 'ف': ' ', 'ل': ' ', 'م

"'Сурков подал в отставку Владислав Сурков 8 мая 2013 года Президент России Владимир Путин подписал Указ «О Суркове В. Ю.», которым отправил чиновника в отставку:  В соответствии с пунктом д статьи 83 Конституции Российской Федерации освободить Суркова Владислава Юрьевича от должности Заместителя Председателя Правительства Российской Федерации Руководителя Аппарата Правительства Российской Федерации по собственному желанию   Пресс-секретарь президента Дмитрий Песков сообщил, что Сурков подал в отставку после совещания с главой государства по исполнению майских указов: Решение президента о подписании заявления связано с тематикой первоочередных задач по реализации майских указов президента и предвыборных обещаний, и функционировании в Белом доме комиссии, которая занималась реализацией этих указов.  Причём в комментариях СМИ он подчёркивал, что Сурков ушёл со своего поста по собственному желанию: Президент удовлетворил просьбу Суркова об увольнении по собственному желанию. Эта формулиро

MAP of OLD TEXT INDICES to NEW TEXT:
{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12, 13: 13, 14: 14, 15: 15, 16: 16, 17: 17, 18: 18, 19: 19, 20: 20, 21: 21, 22: 22, 23: 23, 24: 24, 25: 25, 26: 26, 27: 27, 28: 28, 29: 29, 30: 30, 31: 31, 32: 32, 33: 33, 34: 34, 35: 35, 36: 36, 37: 37, 38: 38, 39: 39, 40: 40, 41: 41, 42: 42, 43: 43, 44: 44, 45: 45, 46: 46, 47: 47, 48: 48, 49: 49, 50: 50, 51: 51, 52: 52, 53: 53, 54: 54, 55: 55, 56: 56, 57: 57, 58: 58, 59: 59, 60: 60, 61: 61, 62: 62, 63: 63, 64: 64, 65: 65, 66: 66, 67: 67, 68: 68, 69: 69, 70: 70, 71: 71, 72: 72, 73: 73, 74: 74, 75: 75, 76: 76, 77: 77, 78: 78, 79: 79, 80: 80, 81: 81, 82: 82, 83: 83, 84: 84, 85: 85, 86: 86, 87: 87, 88: 88, 89: 89, 90: 90, 91: 91, 92: 92, 93: 93, 94: 94, 95: 95, 96: 96, 97: 97, 98: 98, 99: 99, 100: 100, 101: 101, 102: 102, 103: 103, 104: 104, 105: 105, 106: 106, 107: 107, 108: 108, 109: 109, 110: 110, 111: 111, 112: 112, 113: 113, 114: 114, 115: 115, 116: 116, 117: 117, 11

,type,label_and_num,entity,label,num,new_span,extracted,words_extracted_by_wi
0,T1,PERSON 0 6,Сурков,PERSON,"[0, 6]","[0, 0]",Сурков,[Сурков]
1,T2,PERSON 24 40,Владислав Сурков,PERSON,"[24, 40]","[4, 5]",Владислав Сурков,"[Владислав, Сурков]"
2,T4,DATE 41 56,8 мая 2013 года,DATE,"[41, 56]","[6, 9]",8 мая 2013 года,"[8, мая, 2013, года]"
3,T5,PROFESSION 57 66,Президент,PROFESSION,"[57, 66]","[10, 10]",Президент,[Президент]
4,T6,PERSON 74 88,Владимир Путин,PERSON,"[74, 88]","[12, 13]",Владимир Путин,"[Владимир, Путин]"
5,T7,LAW 98 119,Указ «О Суркове В. Ю.,LAW,"[98, 119]","[15, 22]",Указ «О Суркове В. Ю.,"[Указ, «, О, Суркове, В, ., Ю, .]"
6,T8,LAW 104 119,О Суркове В. Ю.,LAW,"[104, 119]","[17, 22]",О Суркове В. Ю.,"[О, Суркове, В, ., Ю, .]"
7,T9,LAW 179 231,пунктом д статьи 83 Конституции Российской Федерации,LAW,"[179, 231]","[34, 40]",пунктом д статьи 83 Конституции Российской Федерации,"[пунктом, д, статьи, 83, Конституции, Российской, Федерации]"
8,T11,PERSON 243 270,Суркова Владислава Юрьевича,PERSON,"[243, 270]","[42, 44]",Суркова Владислава Юрьевича,"[Суркова, Владислава, Юрьевича]"
9,T12,PROFESSION 284 343,Заместителя Председателя Правительства Российской Федерации,PROFESSION,"[284, 343]","[47, 51]",Заместителя Председателя Правительства Российской Федерации,"[Заместителя, Председателя, Правительства, Российской, Федерации]"


Words ['Сурков', 'подал', 'в', 'отставку', 'Владислав', 'Сурков', '8', 'мая', '2013', 'года', 'Президент', 'России', 'Владимир', 'Путин', 'подписал', 'Указ', '«', 'О', 'Суркове', 'В', '.', 'Ю', '.', '»', ',', 'которым', 'отправил', 'чиновника', 'в', 'отставку', ':', 'В', 'соответствии', 'с', 'пунктом', 'д', 'статьи', '83', 'Конституции', 'Российской', 'Федерации', 'освободить', 'Суркова', 'Владислава', 'Юрьевича', 'от', 'должности', 'Заместителя', 'Председателя', 'Правительства', 'Российской', 'Федерации', 'Руководителя', 'Аппарата', 'Правительства', 'Российской', 'Федерации', 'по', 'собственному', 'желанию', 'Пресс', '-', 'секретарь', 'президента', 'Дмитрий', 'Песков', 'сообщил', ',', 'что', 'Сурков', 'подал', 'в', 'отставку', 'после', 'совещания', 'с', 'главой', 'государства', 'по', 'исполнению', 'майских', 'указов', ':', 'Решение', 'президента', 'о', 'подписании', 'заявления', 'связано', 'с', 'тематикой', 'первоочередных', 'задач', 'по', 'реализации', 'майских', 'указов', 'президент

In [ ]:
# dict_one_file

In [ ]:
# !cat '/content/drive/MyDrive/Diploma_paper/NEREL_01/dev/kuleshov.txt' 

Make new files with nerel mrc


In [ ]:
def process_folder(path_dir, folder_name, release=False):
  nerel_names = glob.glob(f"{path_dir}/{folder_name}/*ann")
  for i in range(len(nerel_names)):
    nerel_names[i] = nerel_names[i][:-3]
  res = []
  for name in nerel_names:
    if not release:
      print(name)
    res.append(preprocess_one_file(name, sentence_delimiter=True, take_two_spans = True))
  with open(f'{path_dir}/{folder_name}.json', 'w', encoding='utf8') as f:
    json.dump(res, f, ensure_ascii=False, indent=2)

def process_folder_unique_symbols(path_dir, folder_name, release=False):
  nerel_names = glob.glob(f"{path_dir}/{folder_name}/*ann")
  for i in range(len(nerel_names)):
    nerel_names[i] = nerel_names[i][:-3]
  res = set()
  for name in nerel_names:
    if not release:
      print(name)
    with open(name+'txt') as f:
      text = f.read()
      for t in text:
        res.add(t)
  return res

In [ ]:
path = "/content/drive/MyDrive/Diploma_paper/NEREL_01"
process_folder(path, "dev")
process_folder(path, "test")
process_folder(path, "train")

/content/drive/MyDrive/Diploma_paper/NEREL_01/dev/598012_text.
/content/drive/MyDrive/Diploma_paper/NEREL_01/dev/213805_text.
/content/drive/MyDrive/Diploma_paper/NEREL_01/dev/200774_text.
/content/drive/MyDrive/Diploma_paper/NEREL_01/dev/119336_text.
/content/drive/MyDrive/Diploma_paper/NEREL_01/dev/945932_text.
/content/drive/MyDrive/Diploma_paper/NEREL_01/dev/825235_text.
/content/drive/MyDrive/Diploma_paper/NEREL_01/dev/652538_text.
/content/drive/MyDrive/Diploma_paper/NEREL_01/dev/186860_text.
/content/drive/MyDrive/Diploma_paper/NEREL_01/dev/192856_text.
/content/drive/MyDrive/Diploma_paper/NEREL_01/dev/197352_text.
/content/drive/MyDrive/Diploma_paper/NEREL_01/dev/215419_text.
/content/drive/MyDrive/Diploma_paper/NEREL_01/dev/223960_text.
/content/drive/MyDrive/Diploma_paper/NEREL_01/dev/304703_text.
/content/drive/MyDrive/Diploma_paper/NEREL_01/dev/213124_text.
/content/drive/MyDrive/Diploma_paper/NEREL_01/dev/1277254_text.
/content/drive/MyDrive/Diploma_paper/NEREL_01/dev/1637

In [ ]:
# path = "/content/drive/MyDrive/Diploma_paper/NEREL_01"
# r1 = process_folder_unique_symbols(path, "dev", release=True)
# r2 = process_folder_unique_symbols(path, "test", release=True)
# r3 = process_folder_unique_symbols(path, "train", release=True)
# set.union(r1, r2, r3)

In [ ]:
import os
import json


def convert_file(input_file, output_file, tag2query_file):
    """
    Convert GENIA data to MRC format
    """
    all_data = json.load(open(input_file))
    tag2query = json.load(open(tag2query_file))

    output = []
    origin_count = 0
    new_count = 0

    for data in all_data:
      for elem in data:
          origin_count += 1
          context = elem["context"]
          label2positions = elem["label"]
          for tag_idx, (tag, query) in enumerate(tag2query.items()):
              positions = label2positions.get(tag, [])
              mrc_sample = {
                  "context": context,
                  "query": query,
                  "start_position": [int(x.split(",")[0]) for x in positions],
                  "end_position": [int(x.split(",")[1]) for x in positions],
                  "qas_id": f"{origin_count}.{tag_idx}"
              }
              output.append(mrc_sample)
              new_count += 1

    json.dump(output, open(output_file, "w"), ensure_ascii=False, indent=2)
    print(f"Convert {origin_count} samples to {new_count} samples and save to {output_file}")

for file in ['dev', 'train', 'test']:
  input_file = f'/content/drive/MyDrive/Diploma_paper/NEREL_01/{file}.json'
  output_file = f'/content/drive/MyDrive/Diploma_paper/NEREL_MRC/mrc-ner.{file}'
  tag2query_file = f'/content/drive/MyDrive/Diploma_paper/mrc_preprocess_wanglaiki/mrc-wanglaiki/data_preprocess/queries/ent_nerel.json'
  convert_file(input_file, output_file, tag2query_file)

In [ ]:
tt = '''Двадцать семь кандидатов претендуют на пост президента Мали   В воскресенье в стране состоятся первые президентские выборы после военного путча и последующего перехода северных областей под контроль исламистов и повстанцев-туарегов весной 2012 года.    '''
print(word_tokenizer_span(tt, method='spacy'))
print(tokens_re.findall(tt))

(['Двадцать', 'семь', 'кандидатов', 'претендуют', 'на', 'пост', 'президента', 'Мали', '  ', 'В', 'воскресенье', 'в', 'стране', 'состоятся', 'первые', 'президентские', 'выборы', 'после', 'военного', 'путча', 'и', 'последующего', 'перехода', 'северных', 'областей', 'под', 'контроль', 'исламистов', 'и', 'повстанцев', '-', 'туарегов', 'весной', '2012', 'года', '.', '   '], [(0, 8), (9, 13), (14, 24), (25, 35), (36, 38), (39, 43), (44, 54), (55, 59), (60, 62), (62, 63), (64, 75), (76, 77), (78, 84), (85, 94), (95, 101), (102, 115), (116, 122), (123, 128), (129, 137), (138, 143), (144, 145), (146, 158), (159, 167), (168, 176), (177, 185), (186, 189), (190, 198), (199, 209), (210, 211), (212, 222), (222, 223), (223, 231), (232, 238), (239, 243), (244, 248), (248, 249), (250, 253)])
['Двадцать', 'семь', 'кандидатов', 'претендуют', 'на', 'пост', 'президента', 'Мали', 'В', 'воскресенье', 'в', 'стране', 'состоятся', 'первые', 'президентские', 'выборы', 'после', 'военного', 'путча', 'и', 'последую

In [ ]:
import re
import spacy
from spacy.tokenizer import Tokenizer

emoticons = r'(?:[:;=\^\-oO][\-_\.]?[\)\(\]\[\-DPOp_\^\\\/])'

regex_tweets = [
    emoticons,
    r'<[^>]+>',      ## HTML TAGS
    r'(?:@[\w\d_]+)',   ## @-mentions
    r'(?:\#[\w]+)',  ## #HashTags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
    r"[-+]?(?:\d*\.\d+|\d+)",
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:(?:\d+,?)+(?:\.?\d+)?)',  ##numbers 
    r'(?:[\w_]+)',   #other words
    r'(?:\S)'        ## normal text 
]

tt = '''Двадцать семь кандидатов претендуют на пост президента Мали   В воскресенье в стране состоятся первые президентские выборы после военного путча и последующего перехода северных областей под контроль исламистов и повстанцев-туарегов весной 2012 года.    '''
#compiling regex
tokens_re = re.compile(r'('+'|'.join(regex_tweets)+')' , re.IGNORECASE | re.VERBOSE)

spacyRu.tokenizer = Tokenizer(spacyRu.vocab, infix_finditer=tokens_re.finditer)
doc = spacyRu(tt)

print([t.text for t in doc])

tokens_re.findall(tt)

['"', 'В', 'штате', 'Колорадо', 'полностью', 'отменили', 'рабство', '\n\n', 'Колорадо', 'на', 'карте', 'США', '\n', 'В', 'штате', 'Колорадо', '65', '%', 'избирателей', 'проголосовало', 'за', 'полный', 'запрет', 'рабства', 'и', 'принудительного', 'труда', '.', 'Ранее', 'таковой', 'был', 'разрешён', 'в', 'качестве', 'формы', 'уголовного', 'наказания', '.', '\n\n', 'Ещё', 'в', '1865', 'году', 'была', 'принята', 'Тринадцатая', 'поправка', 'к', 'Конституции', 'США', ',', 'которая', 'запрещала', 'рабство', 'и', 'принудительный', 'труд', ',', 'кроме', 'наказания', 'за', 'преступление', '.', 'В', 'том', 'же', 'году', 'она', 'была', 'ратифицирована', '27', 'штатами', ',', 'что', 'означало', 'вступление', 'её', 'в', 'силу', 'и', 'действие', 'на', 'всей', 'территории', 'США', '.', '\n\n', 'Суть', 'референдума', ',', 'прошедшего', 'в', 'Колорадо', 'в', 'единый', 'день', 'голосования', '6', 'ноября', ',', 'заключалась', 'в', 'том', ',', 'чтобы', 'отредактировать', 'в', 'Конституции', 'штата', 'Коло

['"',
 'В',
 'штате',
 'Колорадо',
 'полностью',
 'отменили',
 'рабство',
 'Колорадо',
 'на',
 'карте',
 'США',
 'В',
 'штате',
 'Колорадо',
 '65',
 '%',
 'избирателей',
 'проголосовало',
 'за',
 'полный',
 'запрет',
 'рабства',
 'и',
 'принудительного',
 'труда',
 '.',
 'Ранее',
 'таковой',
 'был',
 'разрешён',
 'в',
 'качестве',
 'формы',
 'уголовного',
 'наказания',
 '.',
 'Ещё',
 'в',
 '1865',
 'году',
 'была',
 'принята',
 'Тринадцатая',
 'поправка',
 'к',
 'Конституции',
 'США',
 ',',
 'которая',
 'запрещала',
 'рабство',
 'и',
 'принудительный',
 'труд',
 ',',
 'кроме',
 'наказания',
 'за',
 'преступление',
 '.',
 'В',
 'том',
 'же',
 'году',
 'она',
 'была',
 'ратифицирована',
 '27',
 'штатами',
 ',',
 'что',
 'означало',
 'вступление',
 'её',
 'в',
 'силу',
 'и',
 'действие',
 'на',
 'всей',
 'территории',
 'США',
 '.',
 'Суть',
 'референдума',
 ',',
 'прошедшего',
 'в',
 'Колорадо',
 'в',
 'единый',
 'день',
 'голосования',
 '6',
 'ноября',
 ',',
 'заключалась',
 'в',
 'том',